In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
from PIL import Image, ImageTk
import fitz  # PyMuPDF
import threading
import traceback
import re
import os

# Get API key from environment variable or set it here
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "your_groq_api_key_here")  # غَيّرها بمفتاحك

try:
    from groq import Groq
    groq_available = True
except Exception as e:
    groq_available = False
    print("⚠️ groq library not available:", e)


cv_text_global = ""
chat_history = []


def load_cv():
    global cv_text_global
    file_path = filedialog.askopenfilename(
        title="Select CV file",
        filetypes=[("PDF files", "*.pdf"), ("Text files", "*.txt"), ("Word files", "*.docx")]
    )
    if not file_path:
        return

    try:
        # PDF
        if file_path.lower().endswith(".pdf"):
            text = ""
            with fitz.open(file_path) as doc:
                for page in doc:
                    text += page.get_text("text")
        elif file_path.lower().endswith(".docx"):
            try:
                from docx import Document
                doc = Document(file_path)
                paragraphs = [p.text for p in doc.paragraphs]
                text = "\n".join(paragraphs)
            except Exception:
                text = f"[Could not extract text from .docx: {file_path}]"
        # نصي
        else:
            read_success = False
            for enc in ("utf-8", "utf-16", "windows-1256", "iso-8859-6", "latin-1"):
                try:
                    with open(file_path, "r", encoding=enc) as f:
                        text = f.read()
                    read_success = True
                    break
                except Exception:
                    continue
            if not read_success:
                text = "[Failed to read text file with tested encodings]"

        cv_text_global = text
        chat_box.insert(tk.END, f"📄 CV Loaded: {file_path.split('/')[-1]}\n", "bot")
        # أعرض معاينة قصيرة
        preview = text.strip().replace("\r\n", "\n")[:800]
        if preview:
            chat_box.insert(tk.END, "🔎 CV Preview:\n" + preview + ("\n...\n" if len(text) > len(preview) else "\n"), "bot")
        chat_box.yview(tk.END)
        messagebox.showinfo("Success", "✅ CV loaded successfully!")
    except Exception as ex:
        traceback.print_exc()
        messagebox.showerror("Error", f"Failed to load CV:\n{ex}")


def call_groq_api(question, context_text):
    """تستدعى في Thread - تقوم بعمل طلب للموديل وترجع النتيجة"""
    try:
        if not groq_available:
            raise RuntimeError("groq library is not installed.")

        client = Groq(api_key=GROQ_API_KEY)

        context = f"""You are a professional career assistant chatbot.
The user has uploaded their CV below. Use only the CV content to answer the user's question concisely and accurately.
CV Content:
{context_text}
"""
        messages = [
            {"role": "system", "content": context},
            {"role": "user", "content": question}
        ]

        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            temperature=0.2,
            max_completion_tokens=1024
        )

        answer = completion.choices[0].message.content.strip()
        answer = re.sub(r"\*\*((.*?))\*\*", r"((\1))", answer)

        # رجع النتيجة للـ GUI thread
        root.after(0, lambda: display_bot_answer(answer))
    except Exception as e:
        tb = traceback.format_exc()
        print("API call error:", tb)
        root.after(0, lambda: display_bot_answer(f"⚠️ Error calling model: {e}"))


def display_bot_answer(text):
   
    chat_box.insert(tk.END, f"Bot: {text}\n", "bot")
    chat_box.yview(tk.END)
    user_input.delete("1.0", tk.END)

def ask_question():
    if not cv_text_global:
        messagebox.showwarning("Warning", "Upload a CV first!")
        return

    question = user_input.get("1.0", tk.END).strip()
    if not question:
        return

    # عرض سؤال المستخدم في الـ chat_box
    chat_box.insert(tk.END, f"\nYou: {question}\n", "user")
    chat_box.insert(tk.END, "Bot: Thinking...\n", "bot")
    chat_box.yview(tk.END)

    t = threading.Thread(target=call_groq_api, args=(question, cv_text_global), daemon=True)
    t.start()

# =========================
# === واجهة التطبيق GUI ===
# =========================
root = tk.Tk()
root.title("CV Interactive Assistant")
root.geometry("980x720")
root.configure(bg="#F2EDD1")

# ====== تحميل أيقونات بطريقة آمنة ======
def safe_load_image(path, size):
    try:
        img = Image.open(path).convert("RGBA")
        img = img.resize(size)
        return ImageTk.PhotoImage(img)
    except Exception as e:
        print(f"⚠️ couldn't load {path}: {e}")
        return None

upload_icon = safe_load_image("upload.png", (30, 30))
send_icon = safe_load_image("send.png", (22, 22))

# لو الصور اتحملت، نحتفظ بنسخة في root كمرجع دائم
if upload_icon:
    root.upload_icon = upload_icon
if send_icon:
    root.send_icon = send_icon

# ====== Title ======
tk.Label(root, text="CV Chat Assistant", font=("Helvetica", 22, "bold"), bg="#F2EDD1").pack(pady=10)

# ====== Buttons Frame ======
btn_frame = tk.Frame(root, bg="#F2EDD1")
btn_frame.pack(pady=6)

# زر Upload (لو الأيقونة متاحة نحطها ونحتفظ بمراجع)
if upload_icon:
    upload_btn = tk.Button(
        btn_frame,
        text=" Upload CV",
        image=upload_icon,
        compound="left",
        command=load_cv,
        bg="#73C8D2",
        fg="white",
        font=("Helvetica", 12, "bold"),
        width=180,
        height=45,
        relief="raised"
    )
    upload_btn.image = upload_icon
else:
    upload_btn = tk.Button(
        btn_frame,
        text="Upload CV",
        command=load_cv,
        bg="#73C8D2",
        fg="white",
        font=("Helvetica", 12, "bold"),
        width=20,
        height=2,
        relief="raised"
    )
upload_btn.grid(row=0, column=0, padx=10)

# زر إضافي: تحليل تلقائي بسيط بعد رفع الـ CV (مثال)
def quick_audit():
    if not cv_text_global:
        messagebox.showwarning("Warning", "Upload a CV first!")
        return
    # تحليلي بسيط: نبحث عن كلمات مفتاحية شائعة كمثال
    keywords = ["Python", "SQL", "Power BI", "Excel", "Machine Learning", "Java", "C++", "AWS", "Django"]
    present = [k for k in keywords if k.lower() in cv_text_global.lower()]
    missing = [k for k in keywords if k not in present]
    chat_box.insert(tk.END, f"\nBot: Quick audit - found keywords: {', '.join(present) if present else 'None'}\n", "bot")
    chat_box.yview(tk.END)

audit_btn = tk.Button(
    btn_frame,
    text="Quick Audit",
    command=quick_audit,
    bg="#BF092F",
    fg="white",
    font=("Helvetica", 11, "bold"),
    width=12,
    height=2,
    relief="raised"
)
audit_btn.grid(row=0, column=1, padx=6)

# ====== Chat Box ======
chat_box = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=110, height=28, font=("Consolas", 11))
chat_box.pack(padx=12, pady=12, fill="both", expand=True)
chat_box.tag_config("user", foreground="#1E93AB")
chat_box.tag_config("bot", foreground="#280A3E")

# ====== Input Frame ======
input_frame = tk.Frame(root, bg="#F2EDD1")
input_frame.pack(fill="x", padx=12, pady=6)

user_input = tk.Text(input_frame, height=3, width=80, font=("Consolas", 11))
user_input.pack(side="left", padx=6)

# زر Send مع أيقونة إن متاحة
if send_icon:
    send_btn = tk.Button(
        input_frame,
        image=send_icon,
        text=" Send",
        compound="left",
        command=ask_question,
        bg="#1E93AB",
        fg="white",
        font=("Helvetica", 11, "bold"),
        width=120,
        height=36,
        relief="raised"
    )
    send_btn.image = send_icon
else:
    send_btn = tk.Button(
        input_frame,
        text="Send",
        command=ask_question,
        bg="#1E93AB",
        fg="white",
        font=("Helvetica", 11, "bold"),
        width=12,
        height=2,
        relief="raised"
    )
send_btn.pack(side="right", padx=6)

# ====== رسالة بداية ======
chat_box.insert(tk.END, "Bot: Hi! Please upload your CV and ask me any question about it.\n", "bot")

# ====== تشغيل الواجهة ======
root.mainloop()
